In [1]:
import pandas as pd 

#Load and explore new csv data after preprocessing
final_df_train = pd.read_csv("C:/Users/DELL/OneDrive/Desktop/project/skin_cancer_detection-main/notebooks/final_train_data.csv", low_memory=False)
final_df_train.head()


,isic_id,target,patient_id,age_approx,sex,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_location_simple_Torso Front,tbp_lv_location_simple_Unknown,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity,anatom_site_general_nan,tbp_tile_type_3D: XP,tbp_tile_type_3D: white
0,ISIC_0015670,0,IP_1235828,60.0,1.0,3.04,20.244422,16.261975,26.922447,23.954773,...,0,0,0,0,1,0,0,0,0,1
1,ISIC_0015845,0,IP_8170065,60.0,1.0,1.10,31.712570,25.364740,26.331000,24.549290,...,0,0,0,1,0,0,0,0,0,1
2,ISIC_0015864,0,IP_6724798,60.0,1.0,3.40,22.575830,17.128170,37.970460,33.485410,...,0,0,0,0,0,1,0,0,1,0
3,ISIC_0015902,0,IP_4111386,65.0,1.0,3.22,14.242329,12.164757,21.448144,21.121356,...,1,0,1,0,0,0,0,0,1,0
4,ISIC_0024200,0,IP_8313778,55.0,1.0,2.73,24.725520,20.057470,26.464900,25.710460,...,1,0,1,0,0,0,0,0,0,1


In [2]:
import h5py
from sklearn.model_selection import train_test_split
# Load the metadata
# metadata_df = pd.read_csv('train-metadata.csv')
train_hd5 = "train-image.hdf5"

# Load the HDF5 file containing the images
hdf5_file = h5py.File("C:/Users/DELL/OneDrive/Desktop/project/skin_cancer_detection-main/train-image.hdf5")

# Get the image IDs (isic_id)
image_ids = list(hdf5_file.keys())

# Ensure the metadata is consistent with the image IDs
final_df_train = final_df_train[final_df_train['isic_id'].isin(image_ids)]

# Split the dataset into 80% for training and 20% for validation
train_ids, val_ids = train_test_split(final_df_train, test_size=0.2, random_state=42, stratify=final_df_train['target'])  # Stratified split to maintain class balance

In [3]:
final_df_train["target"].value_counts()

target
0    400666
1       393
Name: count, dtype: int64

In [4]:
val_ids.target.value_counts()

target
0    80133
1       79
Name: count, dtype: int64

In [5]:
print(f'train size: {train_ids.shape}, val size: {val_ids.shape}')

train size: (320847, 99), val size: (80212, 99)


In [6]:
import sys
sys.path.append('../src')
sys.path.append('../Models')
from CNN_model import EfficientNetTrainer
# from src import dataloader
from dataloader import ISICDataset
from torch.utils.data import Dataset, DataLoader
from augmentation import augmentation_pipeline


In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

image_size = 224  

transforms_train = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, p=0.75),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),
    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),
    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(image_size, image_size),
    A.CoarseDropout(max_holes=1, max_height=int(image_size * 0.375), max_width=int(image_size * 0.375), fill_value=0, p=0.7),
    ToTensorV2(),
    
])
transformations_valid = A.Compose([
    A.Resize(image_size,image_size ),
    A.Normalize(
            mean=[0.4815, 0.4578, 0.4082], 
            std=[0.2686, 0.2613, 0.2758], 
            max_pixel_value=255.0,
            p=1.0
        ),
    ToTensorV2(),
    ], p=1.)

def create_dataloader(hd5file, csvfile, transform, batch_size=32):
    dataset = ISICDataset(hd5file,csvfile, transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle= True)

train_hd5 = "C:/Users/DELL/OneDrive/Desktop/project/skin_cancer_detection-main/train-image.hdf5"
train_loader = create_dataloader(final_df_train,train_hd5, transforms_train)
# val_loader = create_dataloader(val_ids,train_hd5, transformations_valid)


C:\Users\DELL\AppData\Local\Temp\ipykernel_9604\153597848.py:15: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 30.0)),
C:\Users\DELL\anaconda3\Lib\site-packages\albumentations\core\validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9604\153597848.py:26: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=int(image_size * 0.375), max_width=int(image_size * 0.375), fill_value=0, p=0.7),


In [8]:
train_loader.dataset.targets

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
#trainer = EfficientNetTrainer(num_classes=2, learning_rate=1e-4)
#trainer.train(train_loader, val_loader, epochs=5)
# Assuming you have dataset, groups, and labels ready
trainer = EfficientNetTrainer(num_classes=10, learning_rate=1e-3)
trainer.stratified_k_fold_train(train_loader.dataset, train_loader.dataset.targets, k=5, epochs=5)

#trainer.save_model('efficientnet_best.pth')
#trainer.model

C:\Users\DELL\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DELL\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1/5


C:\Users\DELL\OneDrive\Desktop\project\skin_cancer_detection-main\notebooks\../src\dataloader.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img, dtype=torch.float32)


In [ ]:
trainer.save_model('efficientnet_best.pth')